# Modelo V1

In [10]:
# FLAGS
FLAG_DATASET_DL = True

## 1. Carga de datos y librerías

In [11]:
# Actualizar pip
%pip install --upgrade pip
# Instalar versión específica de numpy
%pip install numpy~=1.26
# Dependencias para graficar y manipular datos
%pip install pandas matplotlib tqdm seaborn
# Dependencias para leer archivos Parquet
%pip install pyarrow fastparquet
# Para predicción
%pip install scikit-surprise scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
# Importar librerías
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Crear directorios
PATH_DATASETS = "datasets"
PATH_DATASETS_FOODCOM = os.path.join(PATH_DATASETS, "foodcom")
PATH_DATASETS_MEALRECPLUS = os.path.join(PATH_DATASETS, "mealrecplus")
PATH_MEALRECPLUS_META_DATA = os.path.join(PATH_DATASETS_MEALRECPLUS, "MealRecPlus-main/MealRec+/MealRec+H/meta_data/")
PATH_MEALRECPLUS_HEALTHINESS = os.path.join(PATH_DATASETS_MEALRECPLUS, "MealRecPlus-main/MealRec+/MealRec+H/healthiness/")
os.makedirs(PATH_DATASETS, exist_ok=True)

In [15]:
if FLAG_DATASET_DL:
  # Descargar "Recipes and Reviews" de Food.com
  os.system(f"curl -L -o {PATH_DATASETS_FOODCOM}.zip https://www.kaggle.com/api/v1/datasets/download/irkaal/foodcom-recipes-and-reviews")
  os.system(f"unzip -o {PATH_DATASETS_FOODCOM}.zip -d {PATH_DATASETS_FOODCOM}")
  # Eliminar el zip
  os.remove(f"{PATH_DATASETS_FOODCOM}.zip")

  # Cita: Dominio público (https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews)

  # Descargar "MealRecPlus" de WUT-IDEA
  os.system(f"curl -L -o {PATH_DATASETS_MEALRECPLUS}.zip https://github.com/WUT-IDEA/MealRecPlus/archive/refs/heads/main.zip")
  os.system(f"unzip -o {PATH_DATASETS_MEALRECPLUS}.zip -d {PATH_DATASETS_MEALRECPLUS}")
  os.system(f"unzip -o {os.path.join(PATH_MEALRECPLUS_META_DATA, 'course')}.zip -d {os.path.join(PATH_MEALRECPLUS_META_DATA, 'course')}")
  os.system(f"unzip -o {os.path.join(PATH_MEALRECPLUS_META_DATA, 'user_course')}.zip -d {os.path.join(PATH_MEALRECPLUS_META_DATA, 'user_course')}")
  # Eliminar el zip
  os.remove(f"{PATH_DATASETS_MEALRECPLUS}.zip")
  os.remove(os.path.join(PATH_MEALRECPLUS_META_DATA, "course.zip"))
  os.remove(os.path.join(PATH_MEALRECPLUS_META_DATA, "user_course.zip"))
  # Cita: Ming Li, Lin Li, Xiaohui Tao, and Jimmy Xiangji Huang. 2024. MealRec+: A Meal Recommendation Dataset with Meal-Course Affiliation for Personal- ization and Healthiness. In Proceedings of the 47th International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR ’24), July 14–18, 2024, Washington, DC, USA. ACM, New York, NY, USA, 11 pages. https://doi.org/10.1145/3626772.3657857 (https://github.com/WUT-IDEA/MealRecPlus)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  723M  100  723M    0     0  23.0M      0  0:00:31  0:00:31 --:--:-- 25.3M


Archive:  datasets/foodcom.zip
  inflating: datasets/foodcom/recipes.csv  
  inflating: datasets/foodcom/recipes.parquet  
  inflating: datasets/foodcom/reviews.csv  
  inflating: datasets/foodcom/reviews.parquet  


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  384M    0  384M    0     0  7680k      0 --:--:--  0:00:51 --:--:-- 33.1M


Archive:  datasets/mealrecplus.zip
8bd7d7d7effb505be4a7d2074b28abad802fe0f7
  inflating: datasets/mealrecplus/MealRecPlus-main/.DS_Store  
  inflating: datasets/mealrecplus/MealRecPlus-main/.gitattributes  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/.DS_Store  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/MealRec+H/.DS_Store  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/MealRec+H/course_category.txt  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/MealRec+H/healthiness/.DS_Store  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/MealRec+H/healthiness/course_fsa.txt  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/MealRec+H/healthiness/course_who.txt  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/MealRec+H/healthiness/meal_fsa.txt  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/MealRec+H/healthiness/meal_who.txt  
  inflating: datasets/mealrecplus/MealRecPlus-main/MealRec+/M

In [7]:
# Foodcom dataframes
df_foodcom_recipes = pd.read_parquet(os.path.join(PATH_DATASETS_FOODCOM, "recipes.parquet"))
df_foodcom_reviews = pd.read_parquet(os.path.join(PATH_DATASETS_FOODCOM, "reviews.parquet"))

In [16]:
# Recetas de Foodcom
df_foodcom_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   RecipeId                    522517 non-null  float64            
 1   Name                        522517 non-null  object             
 2   AuthorId                    522517 non-null  int32              
 3   AuthorName                  522517 non-null  object             
 4   CookTime                    439972 non-null  object             
 5   PrepTime                    522517 non-null  object             
 6   TotalTime                   522517 non-null  object             
 7   DatePublished               522517 non-null  datetime64[us, UTC]
 8   Description                 522512 non-null  object             
 9   Images                      522516 non-null  object             
 10  RecipeCategory              521766 non-null 

In [17]:
# Reviews de Foodcom
df_foodcom_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1401982 entries, 0 to 1401981
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype              
---  ------         --------------    -----              
 0   ReviewId       1401982 non-null  int32              
 1   RecipeId       1401982 non-null  int32              
 2   AuthorId       1401982 non-null  int32              
 3   AuthorName     1401982 non-null  object             
 4   Rating         1401982 non-null  int32              
 5   Review         1401982 non-null  object             
 6   DateSubmitted  1401982 non-null  datetime64[us, UTC]
 7   DateModified   1401982 non-null  datetime64[us, UTC]
dtypes: datetime64[us, UTC](2), int32(4), object(2)
memory usage: 64.2+ MB


In [ ]:
# MealRecPlus dataframes
df_mealrecplus_user_course = pd.read_csv(
    os.path.join(PATH_MEALRECPLUS_META_DATA, "user_course", "user_course.csv"),
    names=["user_id", "course_id", "rating", "dateLastModified"],
    header=None
)
# Convertir la columna 'rating' a tipo numérico, forzando los errores a NaN
df_mealrecplus_user_course['rating'] = pd.to_numeric(df_mealrecplus_user_course['rating'], errors='coerce')
df_mealrecplus_user_course.dropna(subset=['rating'], inplace=True)

# Cargar metadatos de platos
df_mealrecplus_course = pd.read_csv(os.path.join(PATH_MEALRECPLUS_META_DATA, "course", "course.csv"))

# Cargar los índices de usuarios y cursos
df_mealrecplus_user2index = pd.read_csv(os.path.join(PATH_MEALRECPLUS_META_DATA, "user2index.txt"), sep="\t", names=["user_id", "user_index"])
df_mealrecplus_course2index = pd.read_csv(os.path.join(PATH_MEALRECPLUS_META_DATA, "course2index.txt"), sep="\t", names=["course_id", "course_index"])

# Healthiness scores for MealRecPlus
df_mealrecplus_course_fsa = pd.read_csv(os.path.join(PATH_MEALRECPLUS_HEALTHINESS, "course_fsa.txt"), header=None, names=["fsa_score"])
df_mealrecplus_course_who = pd.read_csv(os.path.join(PATH_MEALRECPLUS_HEALTHINESS, "course_who.txt"), header=None, names=["who_score"])

# Puntajes por comida (meal)
df_mealrecplus_meal_fsa = pd.read_csv(os.path.join(PATH_MEALRECPLUS_HEALTHINESS, "meal_fsa.txt"), header=None, names=["fsa_score"])
df_mealrecplus_meal_who = pd.read_csv(os.path.join(PATH_MEALRECPLUS_HEALTHINESS, "meal_who.txt"), header=None, names=["who_score"])

# Puntajes promedio por usuario según sus comidas
df_mealrecplus_user_fsa = pd.read_csv(os.path.join(PATH_MEALRECPLUS_HEALTHINESS, "user_fsa.txt"), header=None, names=["fsa_score"])
df_mealrecplus_user_who = pd.read_csv(os.path.join(PATH_MEALRECPLUS_HEALTHINESS, "user_who.txt"), header=None, names=["who_score"])

/tmp/ipykernel_5928/3656190536.py:2: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mealrecplus_user_course = pd.read_csv(
